## 6.10 Homework

The goal of this homework is to create a tree-based regression model for prediction apartment prices (column `'price'`).

In this homework we'll again use the New York City Airbnb Open Data dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

Let's load the data:

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [3]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

* Apply the log tranform to `price`
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1

In [4]:
df['price'] = np.log1p(df.price)

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price
y_val = df_val.price
y_test = df_test.price

del df_train['price']
del df_val['price']
del df_test['price']

Now, use `DictVectorizer` to turn train and validation into matrices:

In [7]:
from sklearn.feature_extraction import DictVectorizer

In [8]:
train_dict = df_train.to_dict(orient = 'records')
val_dict = df_val.to_dict(orient = 'records')

dv = DictVectorizer(sparse = False)

X_train = dv.fit_transform(train_dict)
X_val = dv.transform(val_dict)

## Question 1

Let's train a decision tree regressor to predict the price variable. 

* Train a model with `max_depth=1`

In [9]:
from sklearn.tree import DecisionTreeRegressor as dt

In [10]:
reg = dt(max_depth =1)

reg.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [11]:
reg.get_params(deep = True)

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': 1,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [12]:
from sklearn.tree import export_text

In [13]:
print(export_text(reg, feature_names = dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [4.29]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [5.15]



Ans: room_type

Which feature is used for splitting the data?

* `room_type`
* `neighbourhood_group`
* `number_of_reviews`
* `reviews_per_month`

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1`  (optional - to make training faster)

In [14]:
from sklearn.ensemble import RandomForestClassifier 

In [15]:
rf = RandomForestClassifier(n_estimators=10, random_state=1)

In [16]:
y_train_int= []
for num in y_train:
    num = int(num)
    y_train_int.append(num)
    
y_train_int

rf.fit(X_train, y_train_int)

RandomForestClassifier(n_estimators=10, random_state=1)

In [17]:
y_pred = rf.predict(X_val)

In [18]:
from sklearn.metrics import mean_squared_error as rmse

In [19]:
rmse(y_val, y_pred)

0.6194266290920285

What's the RMSE of this model on validation?

* 0.059
* 0.259
* 0.459
* 0.659

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10
* Set `random_state` to `1`
* Evaluate the model on the validation dataset

In [20]:
rmse_vals=[]

n_estimators = [10, 20, 30, 40, 50, 60,
                70, 80, 90, 100, 110,
                120, 130, 140, 150,
                160, 170, 180, 190,]

for num in n_estimators:
    
    rf = RandomForestClassifier(n_estimators=num, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train_int)
    
    y_pred = rf.predict(X_val)
    rmse_val =rmse(y_val, y_pred)
    
    print(f'num_estimator: {num}, rmse: {rmse_val.round(3)}')

num_estimator: 10, rmse: 0.619
num_estimator: 20, rmse: 0.58
num_estimator: 30, rmse: 0.561
num_estimator: 40, rmse: 0.554
num_estimator: 50, rmse: 0.557
num_estimator: 60, rmse: 0.557
num_estimator: 70, rmse: 0.552
num_estimator: 80, rmse: 0.553
num_estimator: 90, rmse: 0.552
num_estimator: 100, rmse: 0.548
num_estimator: 110, rmse: 0.549
num_estimator: 120, rmse: 0.547
num_estimator: 130, rmse: 0.548
num_estimator: 140, rmse: 0.549
num_estimator: 150, rmse: 0.545
num_estimator: 160, rmse: 0.546
num_estimator: 170, rmse: 0.547
num_estimator: 180, rmse: 0.546
num_estimator: 190, rmse: 0.544


After which value of `n_estimators` does RMSE stop improving?

- 10
- 50
- 70
- 120

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`

In [39]:
max_depth = [10, 15, 20, 25]

In [40]:
 n_estimators = [10, 20, 30, 40, 50, 60,
                70, 80, 90, 100, 110,
                120, 130, 140, 150,
                160, 170, 180, 190,]

In [44]:
for depth in max_depth:
    print(f'depth: {depth}__________-')     
    for n_estimator in n_estimators:
        
        rf = RandomForestClassifier(n_estimators=n_estimator, random_state=1,max_depth =depth,  n_jobs=-1)
        rf.fit(X_train, y_train_int)
        
        y_pred = rf.predict(X_val)
        rmse_val = rmse(y_val, y_pred)
        
    
        print(f'num_estimator:{n_estimator} rmse:{rmse_val.round(3)}')
        
    print(f'depth: {depth} end _ _ _ __ _ _ _ _ _ -\n')
           
  

depth: 10__________-
num_estimator:10 rmse:0.503
num_estimator:20 rmse:0.5
num_estimator:30 rmse:0.492
num_estimator:40 rmse:0.494
num_estimator:50 rmse:0.493
num_estimator:60 rmse:0.495
num_estimator:70 rmse:0.495
num_estimator:80 rmse:0.497
num_estimator:90 rmse:0.495
num_estimator:100 rmse:0.493
num_estimator:110 rmse:0.492
num_estimator:120 rmse:0.494
num_estimator:130 rmse:0.493
num_estimator:140 rmse:0.493
num_estimator:150 rmse:0.494
num_estimator:160 rmse:0.493
num_estimator:170 rmse:0.493
num_estimator:180 rmse:0.495
num_estimator:190 rmse:0.495
depth: 10 end _ _ _ __ _ _ _ _ _ -

depth: 15__________-
num_estimator:10 rmse:0.542
num_estimator:20 rmse:0.521
num_estimator:30 rmse:0.515
num_estimator:40 rmse:0.514
num_estimator:50 rmse:0.514
num_estimator:60 rmse:0.513
num_estimator:70 rmse:0.511
num_estimator:80 rmse:0.512
num_estimator:90 rmse:0.511
num_estimator:100 rmse:0.511
num_estimator:110 rmse:0.508
num_estimator:120 rmse:0.509
num_estimator:130 rmse:0.511
num_estimator:

What's the best `max_depth`:

* 10
* 15
* 20
* 25

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doint it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [24]:
from sklearn.tree import DecisionTreeClassifier 

In [25]:
rf = RandomForestClassifier(max_depth=20, n_estimators=10, random_state=1)

What's the most important feature? 

* `neighbourhood_group=Manhattan`
* `room_type=Entire home/apt`	
* `longitude`
* `latitude`

In [26]:
rf.fit(X_train, y_train_int)

RandomForestClassifier(max_depth=20, n_estimators=10, random_state=1)

In [27]:
df_train.columns

Index(['neighbourhood_group', 'latitude', 'longitude', 'room_type',
       'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

In [28]:
for feat, importance in zip(df_train.columns, rf.feature_importances_):
    print ('feature: {f}, importance: {i}'.format(f=feat, i=importance))

feature: neighbourhood_group, importance: 0.10815028296010229
feature: latitude, importance: 0.05461524475357107
feature: longitude, importance: 0.17587752485411118
feature: room_type, importance: 0.1893043749397983
feature: minimum_nights, importance: 0.0719037134634464
feature: number_of_reviews, importance: 0.0019921716712088247
feature: reviews_per_month, importance: 0.00700831325014685
feature: calculated_host_listings_count, importance: 0.02502500107846909
feature: availability_365, importance: 0.004808648093854574


Ans: room_type

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

In [29]:
!pip install xgboost

Now change `eta` first to `0.1` and then to `0.01`

In [30]:
import xgboost as xgb

In [31]:
features = dv.get_feature_names()
dtrain = xgb.DMatrix(X_train, label=y_train, feature_names = features)
dval = xgb.DMatrix(X_val, label=y_val, feature_names=features)

In [32]:
watchlist = [(dtrain, 'train'), (dval, 'val')]

In [33]:
xgb_params = {
    'eta': 0.3,
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
        
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200, evals=watchlist,
                 verbose_eval=5 )

[0]	train-rmse:3.02752	val-rmse:3.02415
[5]	train-rmse:0.67490	val-rmse:0.67752
[10]	train-rmse:0.43912	val-rmse:0.44981
[15]	train-rmse:0.42259	val-rmse:0.43827
[20]	train-rmse:0.41716	val-rmse:0.43691
[25]	train-rmse:0.41365	val-rmse:0.43621
[30]	train-rmse:0.40712	val-rmse:0.43543
[35]	train-rmse:0.40444	val-rmse:0.43510
[40]	train-rmse:0.40103	val-rmse:0.43466
[45]	train-rmse:0.39723	val-rmse:0.43371
[50]	train-rmse:0.39446	val-rmse:0.43384
[55]	train-rmse:0.39129	val-rmse:0.43378
[60]	train-rmse:0.38743	val-rmse:0.43404
[65]	train-rmse:0.38421	val-rmse:0.43450
[70]	train-rmse:0.38117	val-rmse:0.43467
[75]	train-rmse:0.37801	val-rmse:0.43489
[80]	train-rmse:0.37668	val-rmse:0.43526
[85]	train-rmse:0.37259	val-rmse:0.43537
[90]	train-rmse:0.36998	val-rmse:0.43539
[95]	train-rmse:0.36742	val-rmse:0.43579
[100]	train-rmse:0.36401	val-rmse:0.43628
[105]	train-rmse:0.36181	val-rmse:0.43673
[110]	train-rmse:0.35892	val-rmse:0.43677
[115]	train-rmse:0.35588	val-rmse:0.43718
[120]	train-rm

In [34]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
        
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200, evals=watchlist,
                 verbose_eval=5 )

[0]	train-rmse:3.87217	val-rmse:3.86889
[5]	train-rmse:2.31905	val-rmse:2.31692
[10]	train-rmse:1.41910	val-rmse:1.41786
[15]	train-rmse:0.91299	val-rmse:0.91348
[20]	train-rmse:0.64528	val-rmse:0.64883
[25]	train-rmse:0.51733	val-rmse:0.52364
[30]	train-rmse:0.46186	val-rmse:0.47101
[35]	train-rmse:0.43843	val-rmse:0.44997
[40]	train-rmse:0.42770	val-rmse:0.44150
[45]	train-rmse:0.42222	val-rmse:0.43795
[50]	train-rmse:0.41868	val-rmse:0.43589
[55]	train-rmse:0.41644	val-rmse:0.43515
[60]	train-rmse:0.41432	val-rmse:0.43460
[65]	train-rmse:0.41226	val-rmse:0.43400
[70]	train-rmse:0.41059	val-rmse:0.43361
[75]	train-rmse:0.40876	val-rmse:0.43336
[80]	train-rmse:0.40747	val-rmse:0.43306
[85]	train-rmse:0.40626	val-rmse:0.43299
[90]	train-rmse:0.40478	val-rmse:0.43280
[95]	train-rmse:0.40406	val-rmse:0.43272
[100]	train-rmse:0.40260	val-rmse:0.43242
[105]	train-rmse:0.40162	val-rmse:0.43244
[110]	train-rmse:0.40016	val-rmse:0.43213
[115]	train-rmse:0.39916	val-rmse:0.43197
[120]	train-rm

In [35]:
xgb_params = {
    'eta': 0.01,
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
        
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=200, evals=watchlist,
                 verbose_eval=5 )

[0]	train-rmse:4.25336	val-rmse:4.25010
[5]	train-rmse:4.04779	val-rmse:4.04454
[10]	train-rmse:3.85242	val-rmse:3.84921
[15]	train-rmse:3.66674	val-rmse:3.66359
[20]	train-rmse:3.49030	val-rmse:3.48719
[25]	train-rmse:3.32263	val-rmse:3.31956
[30]	train-rmse:3.16332	val-rmse:3.16029
[35]	train-rmse:3.01196	val-rmse:3.00898
[40]	train-rmse:2.86817	val-rmse:2.86533
[45]	train-rmse:2.73158	val-rmse:2.72884
[50]	train-rmse:2.60185	val-rmse:2.59925
[55]	train-rmse:2.47865	val-rmse:2.47612
[60]	train-rmse:2.36167	val-rmse:2.35927
[65]	train-rmse:2.25061	val-rmse:2.24835
[70]	train-rmse:2.14519	val-rmse:2.14303
[75]	train-rmse:2.04514	val-rmse:2.04311
[80]	train-rmse:1.95022	val-rmse:1.94827
[85]	train-rmse:1.86015	val-rmse:1.85833
[90]	train-rmse:1.77472	val-rmse:1.77302
[95]	train-rmse:1.69373	val-rmse:1.69214
[100]	train-rmse:1.61695	val-rmse:1.61546
[105]	train-rmse:1.54419	val-rmse:1.54280
[110]	train-rmse:1.47527	val-rmse:1.47404
[115]	train-rmse:1.41000	val-rmse:1.40890
[120]	train-rm

Ans: eta: 0.1

Which eta leads to the best RMSE score on the validation dataset?

* 0.3
* 0.1
* 0.01

## Submit the results


Submit your results here: https://forms.gle/wQgFkYE6CtdDed4w8

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline


The deadline for submitting is 20 October 2021, 17:00 CET (Wednesday). After that, the form will be closed.

